In [1]:
import h5py
import os
import types
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# HOS : A high-order spectral model to solve the Euler equations.

This ipython notebook contains plotting routines to visualize the HOS model output.

The code can be found on **GitHub**: 
- brennanj/hos (master); 
- beisiegel/hos (fork - current)

Questions can be directed to @beisiegel on Github.

#### Some general remarks on the output files

The model produces output files of the form
- data[xxx].1.h5
- data_extra[xxx].1.h5

with [xxx] $\in [0, \mbox{maxtimestep}]$. Leading zeros are left out. The files contain information on the sea surface elevation and potential velocity.

In [ ]:
def PlotSingleFile(DIR, filename):
    f = h5py.File(DIR+fileName, "r")

    # Get the attributes that are contained in the file
    dataset = f.keys()[:]

    print 'Note: Information contained in this dataset: ' + str(dataset)

    size = f[dataset[5]].shape
    time = f[dataset[6]][0]
    
    # discretization (number of grid cells)
    Nx   = f[dataset[2]][0]
    Ny   = f[dataset[3]][0]
    
    # total length of the domain
    Lx   = f[dataset[0]][0]
    Ly   = f[dataset[1]][0]
    
    print Lx, Ly, Nx, Ny

    # Create a grid for plotting phi and eta
    # xx  = np.arange(0,size[0])
    # X,Y = np.meshgrid(xx,xx)
    
    Kx = np.asarray([y * 2. * np.pi/Ly for y in np.arange(-Nx/2,Nx/2-1)])                                                                                                                                                                      
    Ky = np.asarray([y * 2. * np.pi/Ly for y in np.arange(-Ny/2,Ny/2-1)])
    
    x = np.asarray([(Lx / Nx) * x for x in np.arange(0,Nx,1)])
    y = np.asarray([(Ly / Ny) * y for y in np.arange(0,Ny,1)])

    X,Y = np.meshgrid(x,y)

    # Plot of the sea surface elevation
    fig = plt.figure(1)
    plt.contourf(X,Y,f[dataset[4]][:,:])
    plt.xlim(0,Lx)
    plt.ylim(0,Ly)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.colorbar()
    plt.title('Sea surface elevation $\eta$ at time t='+str(time[0]))

    # Plot of the velocity potential
    fig = plt.figure(2)
    plt.contourf(X,Y,f[dataset[5]][:,:])
    plt.xlim(0,Lx)
    plt.ylim(0,Ly)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.colorbar()
    plt.title('Velocity potential $\phi$ at time t='+str(time[0]))

    plt.show()
    f.close()

In [ ]:
# Example Plot

DIR      = "/home/nicole/Development/hos/Csource/2d/"
fileName = "data0.1.h5"

PlotSingleFile(DIR, fileName)

In [ ]:
def PlotAnimation(DIR, keyword):
    idx   = []
    files = []

    if keyword == 'eta':
        val = 4
    else: 
        if keyword == 'phy':
            val = 5
        else:
            print 'Please provide a valid keyword from [eta, phy]'
    
    # Pick and sort all data files
    for file in os.listdir(DIR):
        if (file.startswith('data') & (file[4:7] != '_ex')):
            idx.append(int(float(file[4:7])))
            files.append(file) #(file[4:12])

    files = [x for (y,x) in sorted(zip(idx,files))]

    #plt.ion()
    fig = plt.figure(3)

    for fileName in files:
        f = h5py.File(DIR+fileName, "r")

        # Get the attributes that are contained in the file
        dataset = f.keys()[:]
        time = f[dataset[6]][0]
        size = f[dataset[5]].shape
        
        # Create a grid for plotting phi and eta
        xx  = np.arange(0,size[0])
        X,Y = np.meshgrid(xx,xx)

        plt.clf()
        plt.contourf(X,Y,f[dataset[val]][:,:], np.arange(-.25,.25,.005))
        plt.xlim(0,size[0])
        plt.ylim(0,size[0])
        plt.xlabel('x')
        plt.ylabel('y')

        plt.colorbar() 
        plt.title('Sea surface elevation $\eta$ at time t=' + str(time[0]))

        plt.draw()
        f.close()
    
        plt.show(block=False)    
    plt.close(fig)

In [ ]:
# Example Animation

DIR   = "/home/nicole/Development/hos/Csource/2d/"

PlotAnimation(DIR, 'eta')

## Statistical Evaluations

In [ ]:
def PlotStatistics(DIR):
    
    iBound = 5
    idx    = []
    files  = []
    
    for file in os.listdir(DIR):
        if (file.startswith('data') & (file[4:7] != '_ex')):
            idx.append(int(float(file[4:7])))
            files.append(file) #(file[4:12])

    files = [x for (y,x) in sorted(zip(idx,files))]

    N = len(files)
    #print N
    
    # Initialize vectors
    kurt  = [] #np.zeros(1,N)
    skew  = [] #np.zeros(1,N)
    t     = [] #np.zeros(1,N)
        
    
    for i, fileName in enumerate(files):
        f = h5py.File(DIR+fileName, "r")

        # read in all the data
        dataset = f.keys()[:]
        
        time = f[dataset[6]][0] 
        eta  = f[dataset[4]][:,:] 
        phi  = f[dataset[5]][:,:] 
        Lx   = f[dataset[0]][0] 
        Ly   = f[dataset[1]][0] 
        Nx   = int(f[dataset[2]][0][0])
        Ny   = int(f[dataset[3]][0][0])
        g    = 9.81
    
        edge  = np.arange(-iBound,iBound,0.01)
        gauss = np.exp(np.asarray([-.5*x**2 for x in edge]))
   
        Kx = np.asarray([2 * np.pi / Lx * x for x in np.arange(-Nx / 2,Nx / 2)])
        Ky = np.asarray([2 * np.pi / Ly * y for y in np.arange(-Ny / 2,Ny / 2)])
        x  = np.asarray([Lx / Nx * a for a in np.arange(0,Nx)])
        y  = np.asarray([Ly / Ny * b for b in np.arange(0,Ny)])

        eta_vec = eta.reshape((1,Nx * Ny))

        #i    = nfield+1;  
        t.append(i) 

        mu3     = np.mean(np.power(np.asarray([ ev - np.mean(eta_vec) for ev in eta_vec]),3))
        mu4     = np.mean(np.power(np.asarray([ ev - np.mean(eta_vec) for ev in eta_vec]),4))
        
        # compute kurtosis and skewness of the sea surface displacement eta.
        skew.append(mu3/np.power(np.std(eta_vec),3))
        kurt.append(mu4/np.power(np.std(eta_vec),4))

        # normalization
        eta_vec = (eta_vec - np.mean(eta_vec))/np.std(eta_vec);
        #edge    = min(eta_vec):0.01:max(eta_vec);
        pdf,pdf_edges = np.histogram(eta_vec,edge)
        
        norm_g  = np.asarray([ gs * np.max(pdf) for gs in gauss])
           
        plt.figure(4)
        plt.gca().set_yscale('log')
        plt.plot(edge,norm_g,'r',linewidth=2)
        #hold on
        plt.plot(edge[:len(pdf)],pdf.T,'b')
        
        
        plt.xlim(-5,5)
        plt.ylim(.5,10**4)
        plt.title ('$\eta$ probabilty density function') 
        #legend('Normal Distribution','\eta PDF','location','NorthWest')
        plt.legend()
        plt.draw()

    tvec = np.asarray([elmt * 60. for elmt in t])

    plt.figure(5)
    plt.plot(tvec,kurt,'bd-');
    plt.title ('Kurtosis evolution') 
    plt.xlabel ('time (s)') 
    plt.ylabel ('kurtosis') 

    plt.figure(6)
    plt.plot(tvec,skew,'ro-');
    plt.title ('Skewness evolution') 
    plt.xlabel ('time (s)') 
    plt.ylabel ('skewness') 

        
    plt.show()


In [ ]:
# Example Statistics

DIR   = "/home/nicole/Development/hos/Csource/2d/"

PlotStatistics(DIR)

## Plot (Directional) Spectrum

Compute and plot the spectrum of eta. This is the python script that mimics visualize.m from the HOS.

In [2]:
import FtoD as aux
import OmegaTheta as aux2

In [3]:
def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

In [40]:
def PlotSpectra(DIR, filename):
    f = h5py.File(DIR+fileName, "r")

    # Get the attributes that are contained in the file
    dataset = f.keys()[:]

    size = f[dataset[5]].shape
    time = f[dataset[6]][0]
    
    # discretization (number of grid cells)
    Nx   = f[dataset[2]][0]
    Ny   = f[dataset[3]][0]
    
    # total length of the domain
    Lx   = f[dataset[0]][0]
    Ly   = f[dataset[1]][0]
    
    time = f[dataset[6]][0] 
    eta  = f[dataset[4]][:,:] 
    phi  = f[dataset[5]][:,:] 
    g    = 1 #9.81
    
    Kx = np.asarray([y * 2. * np.pi/Ly for y in np.arange(-Nx/2,Nx/2)])                                                                                                                                                                      
    Ky = np.asarray([y * 2. * np.pi/Ly for y in np.arange(-Ny/2,Ny/2)])
    
    x = np.asarray([(Lx / Nx) * x for x in np.arange(0,Nx,1)])
    y = np.asarray([(Ly / Ny) * y for y in np.arange(0,Ny,1)])

    X,Y = np.meshgrid(x,y)

    # Plot of the spectra
    fig = plt.figure()

    transform_eta = np.fft.fftshift(np.fft.fft2(eta.T))
    transform_phi = np.fft.fftshift(np.fft.fft2(phi.T))
    
    #hetaD = \#FtoD.FtoD(transform_eta, transform_phi, Kx, Ky,g)
    hetaD = aux.FtoD(transform_eta, transform_phi, Kx, Ky,g=g)
    SpectrumD = np.abs(hetaD)
    SpectrumD = SpectrumD.T                                                                                                                                                                                 
                                                                                                                                                       
    Spectrum = abs(np.fft.fftshift(np.fft.fft2(eta)));                                                                                                                                  
    
    vecKx = np.zeros(np.size(Kx))
    vecKy = np.zeros(np.size(Ky))
    
    for m in np.arange(np.size(Kx)):
        vecKx[m]= Kx[m][0]
        vecKy[m]= Ky[m][0]
    
    plt.subplot(3,1,1)  
    plt.loglog(vecKx,Spectrum[int(Ny/2+1),:])  
    plt.xlabel('frequency? (log)')
    plt.ylabel('spectrum (magnitude)(log)')
    plt.draw()     

    plt.subplot(3,1,2)   
    plt.imshow(np.log(Spectrum.T), extent = [vecKx[0], vecKx[len(vecKx)-1], vecKy[0], vecKy[len(vecKy)-1]], aspect=1)
    #imagesc(Kx,Ky,log10(Spectrum))                                                                                                                                                                           
    #set(gca,'YDir','normal');                                                                                                                                                
    #hold on                                                                                                                                       
    #contour(Kx,Ky,log10(Spectrum),[3.8:-0.4:1],'k-')                                                                                                                                      
    #caxis([-5 5]);                                                                                                                                             
    #axis([-3 3 -3 3]);                                                                                                                         
    plt.grid()  
    plt.draw()
    plt.title('Spectrum? (log)')
    plt.colorbar()
    
    plt.subplot(3,1,3, aspect = 'equal')                                                                                                                                                                                           
    #omega, theta = aux2.OmegaTheta(Ky,Kx, g=g)
    omega, theta = aux2.OmegaTheta(Kx, Ky, g=g);      
    X,Y = pol2cart(omega, theta) 
    #X,Y = pol2cart(theta,omega)            
    
    plt.contourf(Y,X,SpectrumD, aspect = 10)                                                                   
    plt.axis([-2, 2, -2, 2])  
    #axis equal                                                                                                                                                                                              
    #axis square                                                                                                                                                                                              
    plt.grid()  
    plt.draw()
    plt.colorbar()
    
    plt.show()
    
    f.close()
    plt.close()
    

In [41]:
# Example Spectra

DIR      = "/home/nicole/Development/hos/Csource/2d/"
fileName = "data0.1.h5"

PlotSpectra(DIR, fileName)